In [1]:
!git clone https://github.com/andssuu/redes_neurais_pos.git

Cloning into 'redes_neurais_pos'...
remote: Enumerating objects: 89, done.
remote: Counting objects: 100% (89/89), done.
remote: Compressing objects: 100% (80/80), done.
remote: Total 89 (delta 17), reused 24 (delta 2), pack-reused 0
Unpacking objects: 100% (89/89), done.


##Multipayer Perceptron (MLP)

Rede Neural baseado no algoritmo de gradiente descendente.  
Os gradientes são calculados usando backpropagation. 

Para mais detalhes, ver os capitulos 13 a 16 do livro no site:

http://deeplearningbook.com.br/

In [2]:
import random
import numpy as np

A entrada é uma lista (`sizes`) contém o número de neurônios nas respectivas camadas da rede. Por exemplo, se a lista for [2, 3, 1] então será uma rede de três camadas, com o primeira camada contendo 2 neurônios, a segunda camada 3 neurônios, e a terceira camada 1 neurônio. Os bias e pesos para a rede são inicializados aleatoriamente, usando uma distribuição Gaussiana com média 0 e variância 1. Note que a primeira camada é assumida como uma camada de entrada, e por convenção não definimos nenhum bias para esses neurônios, pois os bias são usados na computação das saídas das camadas posteriores.


In [5]:
# Função de Ativação Sigmóide
def sigmoid(net):
    return 1.0/(1.0+np.exp(-net))

# Função para retornar as derivadas da função Sigmóide
def sigmoid_prime(z):
    return sigmoid(z)*(1-sigmoid(z))

# Classe Network
class Network(object):

    def __init__(self, sizes, activation_function=sigmoid,
                 prime_function=sigmoid_prime):
        self.num_layers = len(sizes)  #número de neurônios em cada camada
        self.sizes = sizes
        self.biases = [np.random.randn(y, 1) for y in sizes[1:]] #limiar
        self.weights = [np.random.randn(y, x) for x, y in zip(sizes[:-1], sizes[1:])] #pesos
        self.activation_function = activation_function
        self.prime_function = prime_function

    def feedforward(self, x):
        """Retorna a saída da rede z se `x` for entrada."""
        for b, w in zip(self.biases, self.weights):
            x = self.activation_function(np.dot(w, x)+b) #net = (∑xw+b)
        return x

    def SGD(self, training_data, epochs, mini_batch_size, 𝜂, test_data=None):
        """Treinar a rede neural usando o algoritmo mini batch com gradiente descendente. 
         A entrada é uma lista de tuplas
         `(x, y)` representando as entradas de treinamento e as
         saídas. Os outros parâmetros não opcionais são
         auto-explicativos. Se `test_data` for fornecido, então a
         rede será avaliada em relação aos dados do teste após cada
         época e progresso parcial impresso. Isso é útil para
         acompanhar o progresso, mas retarda as coisas substancialmente."""

        #dataset de treino
        training_data = list(training_data)
        n = len(training_data)
        #dataset de teste
        if test_data:
            test_data = list(test_data)
            n_test = len(test_data)

        for j in range(epochs):
            random.shuffle(training_data)
            #técnica que realiza o treinamento por lotes
            #mini_batch_size = tamanho do lote
            mini_batches = [training_data[k:k+mini_batch_size] for k in range(0, n, mini_batch_size)]

            for mini_batch in mini_batches:
                self.update_mini_batch(mini_batch, 𝜂)
            if test_data:
                acc = self.evaluate(test_data)
                print("Epoch {} : {} / {} = {}%".format(j,acc,n_test,(acc*100)/n_test));

            else:
                print("Epoch {} finalizada".format(j))

    def update_mini_batch(self, mini_batch, 𝜂):
        """Atualiza os pesos e limiares da rede aplicando
         a descida do gradiente usando backpropagation para um único mini lote.
         O `mini_batch` é uma lista de tuplas `(x, y)`, e `a` é a taxa de aprendizado."""

        #inicializa matriz com derivadas de pesos e limiares
        nabla_w = [np.zeros(w.shape) for w in self.weights]
        nabla_b = [np.zeros(b.shape) for b in self.biases]

        for x, y in mini_batch:
            #resultado dos deltas do backpropagation sem a multiplicação da taxa de aprendizagem
            #soma os deltas do minibatch
            delta_nabla_b, delta_nabla_w = self.backprop(x, y)
            nabla_w = [nw+dnw for nw, dnw in zip(nabla_w, delta_nabla_w)]
            nabla_b = [nb+dnb for nb, dnb in zip(nabla_b, delta_nabla_b)]

        #atualiza pesos e limiares (𝜂*𝛿*f’(net)*𝑥)  
        self.weights = [w-(𝜂/len(mini_batch))*nw for w, nw in zip(self.weights, nabla_w)]
        self.biases = [b-(𝜂/len(mini_batch))*nb for b, nb in zip(self.biases, nabla_b)]

    def backprop(self, x, y):
        """Retorna uma tupla `(nabla_b, nabla_w)` representando o
         gradiente para a função de custo J_x. `nabla_b` e
         `nabla_w` são listas de camadas de matrizes numpy, semelhantes
         a `self.biases` e `self.weights`."""
        nabla_w = [np.zeros(w.shape) for w in self.weights]
        nabla_b = [np.zeros(b.shape) for b in self.biases]

        # Feedforward
        activation = x

        # Lista para armazenar todas as saídas dos neurônios (z), camada por camada
        activations = [x] 

        # Lista para armazenar todos os vetores net, camada por camada
        nets = [] 

        for b, w in zip(self.biases, self.weights):
            net = np.dot(w, activation)+b   
            nets.append(net)
            activation = self.activation_function(net) #z = valor de saída do neurônio
            activations.append(activation)
        
        # Backward pass 
        
        #última camada -(u-z)f'(net)
        delta = self.cost_derivative(activations[-1], y) * self.prime_function(nets[-1])
        nabla_b[-1] = delta
        nabla_w[-1] = np.dot(delta, activations[-2].transpose()) #(𝑦−𝑧)*f’(net)*𝑥

        # l = 1 significa a última camada de neurônios, l = 2 é a penúltima e assim por diante. 
        for l in range(2, self.num_layers):
            net = nets[-l]
            zs = self.prime_function(net)
            #delta da camada intermediaria. Note que utiliza o delta calculado anteriormente
            delta = np.dot(self.weights[-l+1].transpose(), delta) * zs 
            nabla_b[-l] = delta
            nabla_w[-l] = np.dot(delta, activations[-l-1].transpose()) #∑(𝛿𝑤)f’(net)𝑥
        return (nabla_b, nabla_w)

    def evaluate(self, test_data):
        """Retorna o número de entradas de teste para as quais a rede neural 
         produz o resultado correto. Note que a saída da rede neural
         é considerada o índice de qualquer que seja
         neurônio na camada final que tenha a maior ativação."""
        test_results = [(np.argmax(self.feedforward(x)), y) for (x, y) in test_data]
        return sum(int(x == y) for (x, y) in test_results)

    def cost_derivative(self, output_activations, y):
        """Retorna o vetor das derivadas parciais."""
        return (output_activations-y)


Como exemplo, essa mesma rede será executada na base de dados MNIST. O codigo abaixo carrega a base de dados.

In [6]:
# Carregar o dataset MNIST

# Imports
import pickle
import gzip
import numpy as np

def load_data():
    f = gzip.open('redes_neurais_pos/MLP/mnist.pkl.gz', 'rb')
    training_data, validation_data, test_data = pickle.load(f, encoding="latin1")
    f.close()
    return (training_data, validation_data, test_data)

def load_data_wrapper():
    tr_d, va_d, te_d = load_data()
    training_inputs = [np.reshape(x, (784, 1)) for x in tr_d[0]]
    training_results = [vectorized_result(y) for y in tr_d[1]]
    training_data = zip(training_inputs, training_results)
    validation_inputs = [np.reshape(x, (784, 1)) for x in va_d[0]]
    validation_data = zip(validation_inputs, va_d[1])
    test_inputs = [np.reshape(x, (784, 1)) for x in te_d[0]]
    test_data = zip(test_inputs, te_d[1])
    return (training_data, validation_data, test_data)

def vectorized_result(j):
    e = np.zeros((10, 1))
    e[j] = 1.0
    return e

#Executa a rede neural

Parâmetros de rede:
         2º param é contagem de épocas
         3º param é tamanho do lote
         4º param é a taxa de aprendizado (𝜂)




In [7]:
training_data, validation_data, test_data = load_data_wrapper()
training_data = list(training_data)

#arquitetura da rede
arquitecture = [784, 30, 10]
mlp = Network(arquitecture)
mlp.SGD(training_data, 30, 32, 0.3, test_data=test_data)

Epoch 0 : 4571 / 10000 = 45.71%
Epoch 1 : 6770 / 10000 = 67.7%
Epoch 2 : 7218 / 10000 = 72.18%
Epoch 3 : 7441 / 10000 = 74.41%
Epoch 4 : 7611 / 10000 = 76.11%
Epoch 5 : 7738 / 10000 = 77.38%
Epoch 6 : 7878 / 10000 = 78.78%
Epoch 7 : 7961 / 10000 = 79.61%
Epoch 8 : 8026 / 10000 = 80.26%
Epoch 9 : 8049 / 10000 = 80.49%
Epoch 10 : 8096 / 10000 = 80.96%
Epoch 11 : 8112 / 10000 = 81.12%
Epoch 12 : 8142 / 10000 = 81.42%
Epoch 13 : 8160 / 10000 = 81.6%
Epoch 14 : 8195 / 10000 = 81.95%
Epoch 15 : 8225 / 10000 = 82.25%
Epoch 16 : 8252 / 10000 = 82.52%
Epoch 17 : 8767 / 10000 = 87.67%
Epoch 18 : 8993 / 10000 = 89.93%
Epoch 19 : 9043 / 10000 = 90.43%
Epoch 20 : 9060 / 10000 = 90.6%
Epoch 21 : 9098 / 10000 = 90.98%
Epoch 22 : 9113 / 10000 = 91.13%
Epoch 23 : 9123 / 10000 = 91.23%
Epoch 24 : 9137 / 10000 = 91.37%
Epoch 25 : 9137 / 10000 = 91.37%
Epoch 26 : 9142 / 10000 = 91.42%
Epoch 27 : 9152 / 10000 = 91.52%
Epoch 28 : 9153 / 10000 = 91.53%
Epoch 29 : 9151 / 10000 = 91.51%


##**Mini-Projeto**

A partir dos melhores resultados obtidos no projeto anterior, execute 3 configurações pra cada questão a seguir:

1) Implementar as regularizações L1 e L2

2) Implementar o Momento

3) Comparar os experimentos e explicar o porque de cada resultado? Qual foi a melhor regularização? Por quê? O Momento melhorou os resultados? Por quê? 

Data de Entrega: 14/10/2020
     
      
Complete a Tabela abaixo com os resultados 

\begin{array}{|c|c|c|c|}\hline\\ \\
  Configuracao & 1 & 2 & 3 \\ \hline 
L1 & 92.64 & 93.76 & 93.6  \\ \hline
L2 & 92.55 & 93.16 & 93.61 \\ \hline 
Momento & 94.81 & 95.06 & 59.3 \\ \hline
\end{array}



\\
Data de Entrega: 17/10/2020


**Resolução**

De acordo com o mini-projeto anterior, os melhores resultados foram obtidos nas arquiteturas com as seguintes configurações 

*   Taxa de aprendizagem de 0.5
*   2 Camadas intermediárias
*   Função de ativação sigmóide

Como dito anteriormente, no mini projeto, a única variação foi em relação ao número de neurônios nas camadas intermediárias. 

*   Configuração 1 (90.12%): 28 neurônios em cada camada intermediária
*   Configuração 2 (90.49%): 56 neurônios em cada camada intermediária
*   Configuração 3 (91.46%): 84 neurônios em cada camada intermediária

Implementando regularizações L1 e L2

In [ ]:
class NetworkReg(Network):

    def __init__(self, sizes, activation_function=sigmoid,
                    prime_function=sigmoid_prime, reg_type=1, lamb=0):
        super().__init__(sizes, activation_function=sigmoid,
                    prime_function=sigmoid_prime)
        self.reg_type = reg_type
        self.lamb = lamb

    def update_mini_batch(self, mini_batch, _n):
        """Atualiza os pesos e limiares da rede aplicando
        a descida do gradiente usando backpropagation para um único mini lote.
        O `mini_batch` é uma lista de tuplas `(x, y)`, e `a` é a taxa de aprendizado."""
        # inicializa matriz com derivadas de pesos e limiares
        nabla_w = [np.zeros(w.shape) for w in self.weights]
        nabla_b = [np.zeros(b.shape) for b in self.biases]
        for x, y in mini_batch:
            # resultado dos deltas do backpropagation sem a multiplicação da taxa de aprendizagem
            # soma os deltas do minibatch
            delta_nabla_b, delta_nabla_w = self.backprop(x, y)
            nabla_w = [nw+dnw for nw, dnw in zip(nabla_w, delta_nabla_w)]
            nabla_b = [nb+dnb for nb, dnb in zip(nabla_b, delta_nabla_b)]
        # regulation L1
        if self.reg_type == 1:
            self.weights = [w-(_n/len(mini_batch))*nw+self.lamb/len(mini_batch) for w,
                            nw in zip(self.weights, nabla_w)]
            self.biases = [b-(_n/len(mini_batch))*nb for b,
                           nb in zip(self.biases, nabla_b)]
        # regulation L2
        elif self.reg_type == 2:
            self.weights = [w-(_n/len(mini_batch))*nw+w*self.lamb/len(mini_batch) for w,
                            nw in zip(self.weights, nabla_w)]
            self.biases = [b-(_n/len(mini_batch))*nb for b,
                           nb in zip(self.biases, nabla_b)]

if __name__ == "__main__":
    learning_rate = 0.5
    n_neurons = [[28, 28], [56, 56], [84, 84]]
    _function = [sigmoid, sigmoid_prime]
    _lambda = 5e-06
    for _reg in range(1, 3):
        for _n, conf in enumerate(n_neurons):
            training_data, validation_data, test_data = load_data_wrapper()
            print('Taxa de aprendizagem (𝜂): {}, Configuração {} ('
                'Nº neurônios: {}), Função: {}, Regularização L{}, Lambda: {}'.format(learning_rate, _n+1, conf,
                                                                                        _function[0].__name__, _reg, _lambda))
            arquitecture = [784, conf[0], conf[1], 10]
            mlp = NetworkReg(arquitecture, activation_function=_function[0],
                        prime_function=_function[1], reg_type=_reg, lamb=_lambda)
            mlp.SGD(list(training_data), 30, 32,
                    learning_rate, test_data=test_data)

Taxa de aprendizagem (𝜂): 0.5, Configuração 1 (Nº neurônios: [28, 28]), Função: sigmoid, Regularização L1, Lambda: 5e-06
Epoch 0 : 6192 / 10000 = 61.92%
Epoch 1 : 7650 / 10000 = 76.5%
Epoch 2 : 8210 / 10000 = 82.1%
Epoch 3 : 8490 / 10000 = 84.9%
Epoch 4 : 8647 / 10000 = 86.47%
Epoch 5 : 8762 / 10000 = 87.62%
Epoch 6 : 8821 / 10000 = 88.21%
Epoch 7 : 8877 / 10000 = 88.77%
Epoch 8 : 8948 / 10000 = 89.48%
Epoch 9 : 8978 / 10000 = 89.78%
Epoch 10 : 9003 / 10000 = 90.03%
Epoch 11 : 9028 / 10000 = 90.28%
Epoch 12 : 9048 / 10000 = 90.48%
Epoch 13 : 9067 / 10000 = 90.67%
Epoch 14 : 9096 / 10000 = 90.96%
Epoch 15 : 9118 / 10000 = 91.18%
Epoch 16 : 9114 / 10000 = 91.14%
Epoch 17 : 9144 / 10000 = 91.44%
Epoch 18 : 9148 / 10000 = 91.48%
Epoch 19 : 9161 / 10000 = 91.61%
Epoch 20 : 9175 / 10000 = 91.75%
Epoch 21 : 9202 / 10000 = 92.02%
Epoch 22 : 9193 / 10000 = 91.93%
Epoch 23 : 9202 / 10000 = 92.02%
Epoch 24 : 9227 / 10000 = 92.27%
Epoch 25 : 9237 / 10000 = 92.37%
Epoch 26 : 9241 / 10000 = 92.41%
E

Aplicando a técnica de regularização (L1 e L2), pode-se observar uma melhoria da acurácia em todas as arquiteturas das redes em relação aos experimentos anteriores. Além disso, não houve nenhuma diferença significativa entre as acurácias entre as duas técnicas de regularização.

Implementando técnica de momento com os seguintes parâmetros:

*   Beta: 0.5
*   Número de parcelas: 3



In [19]:
np.warnings.filterwarnings('ignore', category=np.VisibleDeprecationWarning)

class NetworkMoment(Network):

    def __init__(self, sizes, _parcel, _beta, activation_function=sigmoid,
                    prime_function=sigmoid_prime):
        super().__init__(sizes, activation_function=sigmoid,
                    prime_function=sigmoid_prime)
        self.beta = _beta
        self.parcel = _parcel
        self.v = [[np.zeros(w.shape) for w in self.weights] for x in range(_parcel)]

    def SGD(self, training_data, epochs, mini_batch_size, _n, test_data=None):
        training_data = list(training_data)
        n = len(training_data)
        if test_data:
            test_data = list(test_data)
            n_test = len(test_data)
        for j in range(epochs):
            random.shuffle(training_data)
            mini_batches = [training_data[k:k+mini_batch_size]
                            for k in range(0, n, mini_batch_size)]
            for _i, mini_batch in enumerate(mini_batches):
                self.update_mini_batch(mini_batch, _n, _i)
            if test_data:
                acc = self.evaluate(test_data)
                print("Epoch {} : {} / {} = {}%".format(j,
                                                        acc, n_test, (acc*100)/n_test))
            else:
                print("Epoch {} finalizada".format(j))

    def update_mini_batch(self, mini_batch, _n, _i):
        nabla_w = [np.zeros(w.shape) for w in self.weights]
        nabla_b = [np.zeros(b.shape) for b in self.biases]
        for x, y in mini_batch:
            delta_nabla_b, delta_nabla_w = self.backprop(x, y)
            nabla_w = [nw+dnw for nw, dnw in zip(nabla_w, delta_nabla_w)]
            nabla_b = [nb+dnb for nb, dnb in zip(nabla_b, delta_nabla_b)]
        self.v[_i % self.parcel] = [
            _n/len(mini_batch)*nw for nw in nabla_w] + np.array(self.v[(_i % self.parcel)-1])*self.beta
        for _v in self.v:
            self.weights = [w-v for w, v in zip(self.weights, _v)]
        self.biases = [b-(_n/len(mini_batch))*nb for b,
                       nb in zip(self.biases, nabla_b)]

if __name__ == "__main__":
    learning_rate = 0.5
    n_neurons = [[28, 28], [56, 56], [84, 84]]
    _function = [sigmoid, sigmoid_prime]
    beta=0.5
    parcel=3
    for _n, conf in enumerate(n_neurons):
        training_data, validation_data, test_data = load_data_wrapper()
        print('Taxa de aprendizagem (𝜂): {}, Configuração {} ('
            'Nº neurônios: {}), Função: {}, Beta: {}, Nº Parcelas: {}'.format(learning_rate, _n+1, conf,
                                                                                    _function[0].__name__,beta, parcel))
        arquitecture = [784, conf[0], conf[1], 10]
        mlp = NetworkMoment(arquitecture, parcel, beta, activation_function=_function[0],
                    prime_function=_function[1])
        mlp.SGD(list(training_data), 30, 32,
                learning_rate, test_data=test_data)

Taxa de aprendizagem (𝜂): 0.5, Configuração 1 (Nº neurônios: [28, 28]), Função: sigmoid, Beta: 0.5, Nº Parcelas: 3
Epoch 0 : 8751 / 10000 = 87.51%
Epoch 1 : 9052 / 10000 = 90.52%
Epoch 2 : 9150 / 10000 = 91.5%
Epoch 3 : 9234 / 10000 = 92.34%
Epoch 4 : 9275 / 10000 = 92.75%
Epoch 5 : 9299 / 10000 = 92.99%
Epoch 6 : 9330 / 10000 = 93.3%
Epoch 7 : 9353 / 10000 = 93.53%
Epoch 8 : 9379 / 10000 = 93.79%
Epoch 9 : 9362 / 10000 = 93.62%
Epoch 10 : 9387 / 10000 = 93.87%
Epoch 11 : 9410 / 10000 = 94.1%
Epoch 12 : 9418 / 10000 = 94.18%
Epoch 13 : 9403 / 10000 = 94.03%
Epoch 14 : 9456 / 10000 = 94.56%
Epoch 15 : 9426 / 10000 = 94.26%
Epoch 16 : 9460 / 10000 = 94.6%
Epoch 17 : 9441 / 10000 = 94.41%
Epoch 18 : 9438 / 10000 = 94.38%
Epoch 19 : 9464 / 10000 = 94.64%
Epoch 20 : 9473 / 10000 = 94.73%
Epoch 21 : 9478 / 10000 = 94.78%
Epoch 22 : 9483 / 10000 = 94.83%
Epoch 23 : 9447 / 10000 = 94.47%
Epoch 24 : 9477 / 10000 = 94.77%
Epoch 25 : 9458 / 10000 = 94.58%
Epoch 26 : 9452 / 10000 = 94.52%
Epoch 27

Aplicando a técnica de Momento, pode-se observar uma melhoria da acurácia nas duas primerias arquiteturas das redes. Entretanto, na terceira arquitetura houve uma redução significativa da acurácia. Uma boa hipótese para a explicação desse comportamento é a ocorrência de overfitting na rede, uma vez que a terceira arquitetura possui maior quantidade de neurônios nas camadas intermediárias está mais propensa a ocorrência desse problema